### **Semantic Math Types from unstructured text**

This is an overview of the Mathematics Stack Exchange Dataset and a showroom for semantic annotations of mathematical expressions

In [5]:
from mse_db import MSE_DBS # provides an interface for interaction with a local MongoDB instance
from sem_math import PostThread, \
                     FormulaContextType, \
                     FormulaType, \
                     Comparer

The PostThread class encapsulates one question post and all corresponding answers. The three other classes (FormulaContextType, FormulaType and Comparer) are used to determine a semantic type.

#### **1. Dataset Overview**

The basic unit of the processed MSE dataset is a PostThread instance which includes one question post and all corresponding answer posts. All of these are stored within the **posts** attribute. 

![Post Thread Entry in DB](images/post_thread_entry.png)

A **post** entry includes general post metadata, as well as a set of tags and a **PostTypeId** which has the value 1 for question posts and a value of 2 for answer posts.

![Post Entry in DB](images/post_entry.png)

The **Post Thread** entry also includes a formulas attribute, which is a list of all extracted formulas / mathematical expressions. Each formula entry has an id, and a latex string, but also a mathematical type, a relevant string (if found) that describes the value and a decision string that explains how the program determined the type.

![Formula Entry in DB](images/formulas_entry.png)

#### **2. Semantic Types Extraction**

#### **3. Posts titles**